In [1]:
# So get a list of players at the start of a quarter for each time (home 1-5 and away 1-5)
# I think I'll need to loop through each row of the dataframe and modify the list of players for each sub event (home and away)
#     Will also calculate the margin, score, and seconds for each

In [2]:
import requests
import pandas as pd
import numpy as np
import random
import time
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns',500)

In [3]:
# play_by_play_url = 'https://stats.nba.com/stats/playbyplayv2/?gameId=0041700404&&startPeriod=0&endPeriod=14' #
# games_url = 'https://stats.nba.com/stats/leaguegamelog?\
# Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00\
# &PlayerOrTeam=T&Season=2021-22&SeasonType=Playoffs&Sorter=DATE' #
# players_game_url = 'https://stats.nba.com/stats/boxscoreadvancedv2/?gameId=0041700404&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2'

In [4]:
# Headers for querying NBA JSON data api
header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [5]:
# code for getting starting quarter lineups from here: https://github.com/rd11490/NBA-Play-By-Play-Example
# endpoints
def play_by_play_url(game_id):
    return "https://stats.nba.com/stats/playbyplayv2/?gameId={0}&startPeriod=0&endPeriod=14".format(game_id)

def advanced_boxscore_url(game_id, start, end):
    return "https://stats.nba.com/stats/boxscoreadvancedv2/?gameId={0}&startPeriod=0&endPeriod=14&startRange={1}&endRange={2}&rangeType=2".format(game_id, start, end)

def games_url(season,season_type):
    return 'https://stats.nba.com/stats/leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00&PlayerOrTeam=T&Season={}&SeasonType={}&Sorter=DATE'.format(season,season_type)

def players_url(season,season_type,historical=1):
    return 'https://stats.nba.com/stats/playerindex?LeagueID=00&season={}&SeasonType={}&Historical={}'.format(season,season_type,historical)

def player_minutes_url(season,season_type):
    return 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={}&SeasonSegment=&SeasonType={}&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='.format(season,season_type)

def extract_data(url):
    resp = requests.get(url, headers=header_data).json()
#     resp = json.loads(r.data)
    results = resp['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    frame = pd.DataFrame(rows)
    frame.columns = headers
    return frame

def calculate_time_at_period(period):
    if period > 5:
        return (720 * 4 + (period - 5) * (5 * 60)) * 10
    else:
        return (720 * (period - 1)) * 10
    
def split_subs(df, tag):
    subs = df[[tag, 'PERIOD', 'EVENTNUM']]
    subs['SUB'] = tag
    subs.columns = ['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']
    return subs

def get_all_games(season,season_type):
    games = extract_data(games_url(season,season_type))
    games = games[games['MATCHUP'].str.contains('vs.')][['GAME_ID','MATCHUP']]
    games['HOME']=games['MATCHUP'].str.split(' vs. ').str.get(0)
    games['AWAY']=games['MATCHUP'].str.split(' vs. ').str.get(1)
    return games

def get_player_general(season,season_type):
    player_general = extract_data(players_url(season,season_type))
    player_general = player_general[['PLAYER_FIRST_NAME','PLAYER_LAST_NAME','TEAM_ABBREVIATION',
                                     'PERSON_ID',
                                     'POSITION','HEIGHT','WEIGHT','DRAFT_YEAR','FROM_YEAR','TO_YEAR']]
#     player_general['SEASON'] = season
    player_general.rename({'PERSON_ID':'PLAYER_ID'},axis=1,inplace=True)
    return player_general

def get_player_minutes(season,season_type):
    player_minutes = extract_data(player_minutes_url(season,season_type))
    return player_minutes[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','GP','MIN']]


def get_game_data(game_id,games):
    
    frame = extract_data(play_by_play_url(game_id)).fillna('')
    
    substitutionsOnly = frame[frame["EVENTMSGTYPE"] == 8][['PERIOD', 'EVENTNUM', 'PLAYER1_ID', 'PLAYER2_ID']]
    substitutionsOnly.columns = ['PERIOD', 'EVENTNUM', 'OUT', 'IN']

    subs_in = split_subs(substitutionsOnly, 'IN')
    subs_out = split_subs(substitutionsOnly, 'OUT')

    full_subs = pd.concat([subs_out, subs_in], axis=0).reset_index()[['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']]
    first_event_of_period = full_subs.loc[full_subs.groupby(by=['PERIOD', 'PLAYER_ID'])['EVENTNUM'].idxmin()]
    players_subbed_in_at_each_period = first_event_of_period[first_event_of_period['SUB'] == 'IN'][['PLAYER_ID', 'PERIOD', 'SUB']]

    periods = players_subbed_in_at_each_period['PERIOD'].drop_duplicates().values.tolist()

    frames = []
    for period in periods:

        low = calculate_time_at_period(period) + 5
        high = calculate_time_at_period(period + 1) - 5
        boxscore = advanced_boxscore_url(game_id, low, high)
        boxscore_players = extract_data(boxscore)[['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION']]
        boxscore_players['PERIOD'] = period

        players_subbed_in_at_period = players_subbed_in_at_each_period[players_subbed_in_at_each_period['PERIOD'] == period]

        joined_players = pd.merge(boxscore_players, players_subbed_in_at_period, on=['PLAYER_ID', 'PERIOD'], how='left')
        joined_players = joined_players[pd.isnull(joined_players['SUB'])][['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ABBREVIATION', 'PERIOD']]
        frames.append(joined_players)

    out = pd.concat(frames)    
    out = out.groupby(['TEAM_ABBREVIATION','PERIOD'])['PLAYER_ID'].apply(lambda x: ','.join(map(str, x))).reset_index()
    
    frame['HOME']=games[games['GAME_ID']==game_id]['HOME'].values[0]
    frame['AWAY']=games[games['GAME_ID']==game_id]['AWAY'].values[0]
    frame = frame.merge(out,how='left',left_on=['HOME','PERIOD'],right_on=['TEAM_ABBREVIATION','PERIOD']).drop('TEAM_ABBREVIATION',axis=1)
    frame.rename({'PLAYER_ID':'HOME_PLAYER_IDS'},axis=1,inplace=2)
    frame = frame.merge(out,how='left',left_on=['AWAY','PERIOD'],right_on=['TEAM_ABBREVIATION','PERIOD']).drop('TEAM_ABBREVIATION',axis=1)
    frame.rename({'PLAYER_ID':'AWAY_PLAYER_IDS'},axis=1,inplace=2)
    frame['HOME_SUB']=frame['HOMEDESCRIPTION'].str.contains('SUB')
    frame['AWAY_SUB']=frame['VISITORDESCRIPTION'].str.contains('SUB')
    frame['QUARTER_START']=frame['NEUTRALDESCRIPTION'].str.contains('Start')
    frame['PLAYER_CHANGE']=np.where((frame['QUARTER_START']==True)|(frame['AWAY_SUB']==True)|(frame['HOME_SUB']==True),True,False)
    frame['HOME_PLAYER_IDS']=np.where(frame['QUARTER_START']==True,frame['HOME_PLAYER_IDS'],np.nan)
    frame['AWAY_PLAYER_IDS']=np.where(frame['QUARTER_START']==True,frame['AWAY_PLAYER_IDS'],np.nan)
    
    for idx, row in frame.iterrows():
        if idx == 0 or row['QUARTER_START']==True:
            continue
        else:
            frame.at[idx,'AWAY_PLAYER_IDS']=frame.at[idx-1,'AWAY_PLAYER_IDS']
            frame.at[idx,'HOME_PLAYER_IDS']=frame.at[idx-1,'HOME_PLAYER_IDS']
            if row['AWAY_SUB']==True:
                frame.at[idx,'AWAY_PLAYER_IDS']=frame.at[idx,'AWAY_PLAYER_IDS'].replace(str(row['PLAYER1_ID']),str(row['PLAYER2_ID']))
            if row['HOME_SUB']==True:
                frame.at[idx,'HOME_PLAYER_IDS']=frame.at[idx,'HOME_PLAYER_IDS'].replace(str(row['PLAYER1_ID']),str(row['PLAYER2_ID']))
    frame['HOME_PLAYER_IDS'].ffill(inplace=True)
    frame['AWAY_PLAYER_IDS'].ffill(inplace=True)
    
    frame=frame[frame['PLAYER_CHANGE']==False]
    frame=frame[frame['NEUTRALDESCRIPTION']=='']
    frame=frame[(~frame['HOMEDESCRIPTION'].str.contains('Jump Ball'))&(~frame['VISITORDESCRIPTION'].str.contains('Jump Ball'))]
    frame=frame[(~frame['HOMEDESCRIPTION'].str.contains('Timeout'))&(~frame['VISITORDESCRIPTION'].str.contains('Timeout'))]
    
    frame['NEXT_HOME_PLAY']=frame['HOMEDESCRIPTION'].shift(-1)
    frame['NEXT_AWAY_PLAY']=frame['VISITORDESCRIPTION'].shift(-1)
    frame['TIMER']=frame['PCTIMESTRING'].str.split(':').str.get(0).astype(int)+frame['PCTIMESTRING'].str.split(':').str.get(1).astype(int)/60
    frame['SECONDS_PASSED']=np.where(frame['TIMER'].shift(1)>=frame['TIMER'],frame['TIMER'].shift(1)*60-frame['TIMER']*60,(np.ceil(frame['TIMER'])-frame['TIMER'])*60)
    frame['HOME_POSSESSION_FLAG']=np.where((frame['NEXT_HOME_PLAY']=='')&(frame['HOMEDESCRIPTION']!=''),1,0)
    frame['AWAY_POSSESSION_FLAG']=np.where((frame['NEXT_AWAY_PLAY']=='')&(frame['VISITORDESCRIPTION']!=''),1,0)
    
    frame['AWAY_SCORE']=frame['SCORE'].str.split(' - ').str.get(0)
    frame['AWAY_SCORE']=np.where(frame['AWAY_SCORE']=='',np.nan,frame['AWAY_SCORE'])
    frame['AWAY_SCORE'].ffill(inplace=True)
    frame['AWAY_SCORE'].fillna(0,inplace=True)
    frame['HOME_SCORE']=frame['SCORE'].str.split(' - ').str.get(1)
    frame['HOME_SCORE'].ffill(inplace=True)
    frame['HOME_SCORE'].fillna(0,inplace=True)
    frame['HOME_SCORE_DIFFERENCE']=frame['HOME_SCORE'].astype(int)-frame['HOME_SCORE'].astype(int).shift(1)
    frame['AWAY_SCORE_DIFFERENCE']=frame['AWAY_SCORE'].astype(int)-frame['AWAY_SCORE'].astype(int).shift(1)
    frame['HOME_SCORE_DIFFERENCE'].fillna(0,inplace=True)
    frame['AWAY_SCORE_DIFFERENCE'].fillna(0,inplace=True)
    frame['HOME_VS_AWAY']=frame['HOME_SCORE_DIFFERENCE']-frame['AWAY_SCORE_DIFFERENCE']
    return frame

In [6]:
season = '2021-22'
season_type = 'Regular+Season'
games = get_all_games(season,season_type)
games

,GAME_ID,MATCHUP,HOME,AWAY
0,0022101224,MIN vs. CHI,MIN,CHI
2,0022101228,PHI vs. DET,PHI,DET
4,0022101223,MEM vs. BOS,MEM,BOS
7,0022101220,DEN vs. LAL,DEN,LAL
8,0022101227,ORL vs. MIA,ORL,MIA
...,...,...,...,...
2450,0022100006,TOR vs. WAS,TOR,WAS
2452,0022100004,DET vs. CHI,DET,CHI
2455,0022100003,CHA vs. IND,CHA,IND
2457,0022100001,MIL vs. BKN,MIL,BKN


In [7]:
players = get_player_general(season,season_type)
players

,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,TEAM_ABBREVIATION,PLAYER_ID,POSITION,HEIGHT,WEIGHT,DRAFT_YEAR,FROM_YEAR,TO_YEAR
0,Alaa,Abdelnaby,POR,76001,F,6-10,240,1990.0,1990,1994
1,Zaid,Abdul-Aziz,HOU,76002,C,6-9,235,1968.0,1968,1977
2,Kareem,Abdul-Jabbar,LAL,76003,C,7-2,225,1969.0,1969,1988
3,Mahmoud,Abdul-Rauf,DEN,51,G,6-1,162,1990.0,1990,2000
4,Tariq,Abdul-Wahad,SAC,1505,F-G,6-6,235,1997.0,1997,2003
...,...,...,...,...,...,...,...,...,...,...
4726,Ante,Zizic,CLE,1627790,C,6-10,266,2016.0,2017,2019
4727,Jim,Zoet,DET,78647,C,7-1,240,NaN,1982,1982
4728,Bill,Zopf,MIL,78648,G,6-1,170,1970.0,1970,1970
4729,Ivica,Zubac,LAC,1627826,C,7-0,240,2016.0,2016,2021


In [8]:
player_minutes = get_player_minutes(season,season_type)
player_minutes

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN
0,203932,Aaron Gordon,DEN,75,2375.418333
1,1630565,Aaron Henry,PHI,6,16.983333
2,1628988,Aaron Holiday,PHX,63,1020.750000
3,1630174,Aaron Nesmith,BOS,52,573.878333
4,1630598,Aaron Wiggins,OKC,50,1208.750000
...,...,...,...,...,...
600,203897,Zach LaVine,CHI,67,2328.078333
601,1630285,Zavier Simpson,OKC,4,174.083333
602,1630192,Zeke Nnaji,DEN,41,697.608333
603,1630533,Ziaire Williams,MEM,62,1346.466667


In [9]:
player_minutes_low = player_minutes[(player_minutes['GP']<10)|(player_minutes['MIN']<200)]
player_minutes_low

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN
1,1630565,Aaron Henry,PHI,6,16.983333
5,1627846,Abdel Nader,PHX,14,145.238333
6,1630278,Ade Murkey,SAC,1,1.466667
8,1629958,Ahmad Caver,IND,1,0.843333
11,1630758,Aleem Ford,ORL,5,73.530000
...,...,...,...,...,...
594,1629875,Xavier Moon,LAC,10,137.368333
595,1630270,Xavier Sneed,UTA,9,38.716667
598,1630582,Yves Pons,MEM,12,70.975000
601,1630285,Zavier Simpson,OKC,4,174.083333


In [10]:
# # Run the dataframe all together
# for i,game in enumerate(games['GAME_ID'].values[0:10]):
#     if i == 0:
#         results = get_game_data(game_id=game,games=games)
#     else:
#         results = pd.concat([results,(get_game_data(game_id=game,games=games))],axis=1)
# results

In [11]:
now_time = str(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())).replace(' ','-').replace(':','-')

In [ ]:
problem_games = []
completed_games = []
for i,game in enumerate(games['GAME_ID'].values):
    if i == 0:
        results = get_game_data(game_id=game,games=games)
        results.to_csv('play_by_play_run'+now_time+'.csv')
        completed_games.append(game)
    else:
        try:
            results = get_game_data(game_id=game,games=games)
            completed_games.append(game)
            results.to_csv('play_by_play_run'+now_time+'.csv', mode='a', header=False)
        except Exception as e:
            print('Problem with run ' + str(i+1) + ', game id ' + str(game))
            print(e)
            problem_games.append(game)
        if (i+1)%10 == 0:
            print('Run ' + str(i+1) + ' completed. ' + str(len(games['GAME_ID'])-i+1) + ' to go.')
    # Wait time to not get banned from API
    time.sleep(random.randint(5,10))
print('Done with the errors in the problem_games list.')

In [ ]:
i = 0
for game in problem_games[:]:
    i+=1
    try:
        results = get_game_data(game_id=game,games=games)
        completed_games.append(game)
        results.to_csv('play_by_play_run'+now_time+'.csv', mode='a', header=False)
        problem_games.remove(game)
    except Exception as e:
        print('Problem with run ' + str(i+1) + ', game id ' + str(game))
        print(e)
    if (i+1)%10 == 0:
        print('Run ' + str(i+1) + ' completed. ' + str(len(problem_games)-i+1) + ' to go.')
    else:
        continue
    # Wait time to not get banned from API
    time.sleep(random.randint(5,10))
print('Done with the errors in the problem_games list.')

In [ ]:
problem_games

In [58]:
test_df = pd.read_csv('play_by_play_run2022-05-17-13-58-05.csv')
test_df['HOME_VS_AWAY']=np.where((test_df['AWAY_SCORE_DIFFERENCE']<0)|(test_df['HOME_SCORE_DIFFERENCE']<0),0,test_df['HOME_VS_AWAY'])
test_df['AWAY_SCORE_DIFFERENCE']=np.where(test_df['AWAY_SCORE_DIFFERENCE']<0,0,test_df['AWAY_SCORE_DIFFERENCE'])
test_df['HOME_SCORE_DIFFERENCE']=np.where(test_df['HOME_SCORE_DIFFERENCE']<0,0,test_df['HOME_SCORE_DIFFERENCE'])
test_df['HOME_PLAYER_IDS']=test_df['HOME_PLAYER_IDS'].str.split(',')
test_df['AWAY_PLAYER_IDS']=test_df['AWAY_PLAYER_IDS'].str.split(',')
test_df['HOME_PLAYER_IDS']=test_df['HOME_PLAYER_IDS'].apply(sorted)
test_df['AWAY_PLAYER_IDS']=test_df['AWAY_PLAYER_IDS'].apply(sorted)
test_df['HOME_PLAYER_IDS']=test_df['HOME_PLAYER_IDS'].apply(','.join)
test_df['AWAY_PLAYER_IDS']=test_df['AWAY_PLAYER_IDS'].apply(','.join)
# test_df[['HOME_PLAYER_IDS','AWAY_PLAYER_IDS']]=test_df[['HOME_PLAYER_IDS','AWAY_PLAYER_IDS']].apply
test_df = test_df.groupby(['HOME_PLAYER_IDS','AWAY_PLAYER_IDS']).agg({'SECONDS_PASSED':'sum','HOME_POSSESSION_FLAG':'sum',
                                                           'AWAY_POSSESSION_FLAG':'sum','HOME_SCORE_DIFFERENCE':'sum',
                                                           'AWAY_SCORE_DIFFERENCE':'sum','HOME_VS_AWAY':'sum'}).reset_index()
test_df['HOME_VS_AWAY_PER_POSSESSION']=test_df['HOME_VS_AWAY']/(0.5*test_df['AWAY_POSSESSION_FLAG']+0.5*test_df['HOME_POSSESSION_FLAG'])
test_df['HOME_VS_AWAY_PER_POSSESSION'].replace([np.inf,-np.inf,np.nan],0,inplace=True)
test_df['HOME_PLAYER_IDS']=test_df['HOME_PLAYER_IDS'].str.split(',')
test_df['AWAY_PLAYER_IDS']=test_df['AWAY_PLAYER_IDS'].str.split(',')
home_explode = test_df.explode('HOME_PLAYER_IDS').reset_index()
home_explode['HOME_PLAYER_IDS']=home_explode['HOME_PLAYER_IDS'].astype(int)
drop_home = player_minutes_low.merge(home_explode,how='inner',left_on='PLAYER_ID',right_on='HOME_PLAYER_IDS')
away_explode = test_df.explode('AWAY_PLAYER_IDS').reset_index()
away_explode['AWAY_PLAYER_IDS']=away_explode['AWAY_PLAYER_IDS'].astype(int)
drop_away = player_minutes_low.merge(away_explode,how='inner',left_on='PLAYER_ID',right_on='AWAY_PLAYER_IDS')
test_df=test_df[(~test_df.index.isin(drop_home['index']))&(~test_df.index.isin(drop_away['index']))]
cols = test_df.columns
mlb = MultiLabelBinarizer()
test_df = test_df.merge(pd.DataFrame(mlb.fit_transform(test_df['HOME_PLAYER_IDS']),columns=mlb.classes_, index=test_df.index),
                     how='inner', left_index=True, right_index=True)
for idx,row in test_df.iterrows():
    for player in row['AWAY_PLAYER_IDS']:
        test_df.at[idx,player]=-1
test_df.fillna(0,inplace=True)
# test_df.to_csv('test_df'+now_time+'.csv')
test_df.sort_values(by='HOME_VS_AWAY')

,HOME_PLAYER_IDS,AWAY_PLAYER_IDS,SECONDS_PASSED,HOME_POSSESSION_FLAG,AWAY_POSSESSION_FLAG,HOME_SCORE_DIFFERENCE,AWAY_SCORE_DIFFERENCE,HOME_VS_AWAY,HOME_VS_AWAY_PER_POSSESSION,101108,101150,1626145,1626147,1626149,1626153,1626156,1626157,1626158,1626159,1626162,1626164,1626166,1626167,1626168,1626169,1626171,1626172,1626174,1626179,1626181,1626192,1626195,1626196,1626204,1626208,1626220,1626224,1627734,1627736,1627737,1627739,1627741,1627742,1627745,1627747,1627749,1627751,1627752,1627756,1627759,1627761,1627763,1627774,1627777,1627780,1627783,1627788,1627789,1627814,1627823,1627826,1627827,1627832,1627854,1627863,1627884,1627936,1628021,1628035,1628365,1628366,1628367,1628368,1628369,1628370,1628372,1628373,1628374,1628378,1628379,1628380,1628381,1628384,1628386,1628389,1628392,1628396,1628398,1628400,1628401,1628402,1628404,1628408,1628415,1628418,1628420,1628425,1628432,1628449,1628464,1628467,1628470,1628539,1628960,1628963,1628964,1628966,1628969,1628970,1628971,1628972,1628973,1628975,1628976,1628977,1628978,1628983,1628984,1628988,1628989,1628991,1628995,1628997,1628998,1629001,1629002,1629003,1629004,1629006,1629011,1629012,1629013,1629014,1629018,1629020,1629021,1629022,1629023,1629026,1629027,1629028,1629029,1629048,1629052,1629056,1629057,1629060,1629103,1629109,1629111,1629117,1629130,1629139,1629162,1629164,1629185,1629216,1629234,1629308,1629599,1629604,1629611,1629620,1629622,1629628,1629629,1629630,1629631,1629632,1629633,1629634,1629636,1629637,1629638,1629639,1629640,1629641,1629642,1629643,1629644,1629645,1629647,1629649,1629650,1629651,1629652,1629655,1629656,1629659,1629660,1629661,1629667,1629669,1629670,1629672,1629673,1629675,1629676,1629678,1629680,1629681,1629684,1629685,1629717,1629723,1629726,1629731,1629744,1629750,1629833,1630162,1630163,1630165,1630166,1630167,1630168,1630169,1630170,1630171,1630172,1630173,1630174,1630175,1630177,1630178,1630180,1630181,1630182,1630183,1630184,1630188,1630190,1630191,1630192,1630193,1630194,1630195,1630197,1630198,1630200,1630201,1630202,1630205,1630208,1630209,1630214,1630215,1630217,1630219,1630224,1630227,1630228,1630230,1630231,1630233,1630235,1630240,1630245,1630249,1630256,1630264,1630267,1630314,1630322,1630526,1630527,1630528,1630529,1630530,1630532,1630533,1630535,1630537,1630538,1630540,1630541,1630543,1630544,1630547,1630549,1630550,1630551,1630553,1630556,1630557,1630558,1630559,1630560,1630561,1630563,1630567,1630568,1630570,1630572,1630578,1630579,1630580,1630581,1630583,1630586,1630587,1630591,1630595,1630596,1630598,1630613,1630625,1630631,1630678,1630686,1630688,1630792,1630846,200746,200752,200765,200768,200782,200794,201142,201143,201144,201145,201152,201565,201566,201567,201568,201569,201571,201572,201577,201580,201586,201587,201588,201599,201609,201933,201935,201937,201939,201942,201949,201950,201959,201961,201976,201980,201988,202066,202083,202324,202326,202330,202331,202334,202339,202340,202355,202357,202362,202397,202681,202683,202684,202685,202687,202689,202691,202692,202693,202694,202696,202699,202704,202709,202710,202711,202722,202738,202954,203076,203078,203081,203082,203083,203084,203085,203087,203090,203095,203107,203109,203110,203114,203115,203145,203200,203210,203457,203458,203463,203468,203469,203471,203473,203476,203482,203484,203486,203488,203490,203493,203496,203497,203500,203501,203503,203504,203507,203526,203552,203585,203648,203897,203901,203903,203914,203915,203918,203920,203924,203925,203926,203932,203935,203937,203939,203944,203952,203954,203991,203992,203994,203999,204001,204025,204060,204456,2544,2546,2730,2738,2772
6653,"[1626174, 1628464, 1629645, 1630224, 1630256]","[101108, 1626164, 1628969, 201580, 203109]",886.0,31,32,16.0,41.0,-25.0,-0.793651,-1,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [59]:
# This was crashing until I made a new Conda env, re-installing my libraries
lr = LinearRegression()
lr.fit(test_df.drop(cols,axis=1),test_df['HOME_VS_AWAY_PER_POSSESSION'])
linreg_results = pd.DataFrame({'coef':lr.coef_, 'player':test_df.drop(cols,axis=1).columns}).sort_values(by = 'coef', ascending = False)
linreg_results['player']=linreg_results['player'].astype(int)
linreg_results_player = linreg_results.merge(player_minutes,how='inner',left_on='player',right_on='PLAYER_ID')
linreg_results_player

,coef,player,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN
0,0.331206,1630587,1630587,Isaiah Livers,DET,19,383.256667
1,0.322597,1630264,1630264,Anthony Gill,WAS,44,463.400000
2,0.312775,1630586,1630586,Usman Garuba,HOU,24,239.160000
3,0.257247,203991,203991,Clint Capela,ATL,74,2041.530000
4,0.256425,1630631,1630631,Jose Alvarado,NOP,54,833.908333
...,...,...,...,...,...,...,...
441,-0.228038,1627756,1627756,Denzel Valentine,UTA,24,221.996667
442,-0.236899,1630240,1630240,Saben Lee,DET,37,603.728333
443,-0.248862,1629644,1629644,KZ Okpala,MIA,21,244.071667
444,-0.343550,1630686,1630686,Georgios Kalaitzakis,OKC,13,213.478333


In [61]:
linreg_results_player[linreg_results_player['MIN']>=500].head(50)

,coef,player,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN
3,0.257247,203991,203991,Clint Capela,ATL,74,2041.530000
4,0.256425,1630631,1630631,Jose Alvarado,NOP,54,833.908333
8,0.235905,1627742,1627742,Brandon Ingram,NOP,55,1869.181667
9,0.232666,1627936,1627936,Alex Caruso,CHI,41,1146.831667
10,0.232355,203994,203994,Jusuf Nurkic,POR,56,1578.485000
11,0.226758,201950,201950,Jrue Holiday,MIL,67,2207.440000
12,0.226543,1630532,1630532,Franz Wagner,ORL,79,2429.046667
13,0.226506,201588,201588,George Hill,MIL,54,1253.203333
14,0.225376,1626167,1626167,Myles Turner,IND,42,1234.728333
15,0.221737,201939,201939,Stephen Curry,GSW,64,2211.085000


In [78]:
results = test_df.corr().reset_index()
results

,index,SECONDS_PASSED,HOME_POSSESSION_FLAG,AWAY_POSSESSION_FLAG,HOME_SCORE_DIFFERENCE,AWAY_SCORE_DIFFERENCE,HOME_VS_AWAY,HOME_VS_AWAY_PER_POSSESSION,101108,101150,1626145,1626147,1626149,1626153,1626156,1626157,1626158,1626159,1626162,1626164,1626166,1626167,1626168,1626169,1626171,1626172,1626174,1626179,1626181,1626192,1626195,1626196,1626204,1626208,1626220,1626224,1627734,1627736,1627737,1627739,1627741,1627742,1627745,1627747,1627749,1627751,1627752,1627756,1627759,1627761,1627763,1627774,1627777,1627780,1627783,1627788,1627789,1627814,1627823,1627826,1627827,1627832,1627854,1627863,1627884,1627936,1628021,1628035,1628365,1628366,1628367,1628368,1628369,1628370,1628372,1628373,1628374,1628378,1628379,1628380,1628381,1628384,1628386,1628389,1628392,1628396,1628398,1628400,1628401,1628402,1628404,1628408,1628415,1628418,1628420,1628425,1628432,1628449,1628464,1628467,1628470,1628539,1628960,1628963,1628964,1628966,1628969,1628970,1628971,1628972,1628973,1628975,1628976,1628977,1628978,1628983,1628984,1628988,1628989,1628991,1628995,1628997,1628998,1629001,1629002,1629003,1629004,1629006,1629011,1629012,1629013,1629014,1629018,1629020,1629021,1629022,1629023,1629026,1629027,1629028,1629029,1629048,1629052,1629056,1629057,1629060,1629103,1629109,1629111,1629117,1629130,1629139,1629162,1629164,1629185,1629216,1629234,1629308,1629599,1629604,1629611,1629620,1629622,1629628,1629629,1629630,1629631,1629632,1629633,1629634,1629636,1629637,1629638,1629639,1629640,1629641,1629642,1629643,1629644,1629645,1629647,1629649,1629650,1629651,1629652,1629655,1629656,1629659,1629660,1629661,1629667,1629669,1629670,1629672,1629673,1629675,1629676,1629678,1629680,1629681,1629684,1629685,1629717,1629723,1629726,1629731,1629744,1629750,1629833,1630162,1630163,1630165,1630166,1630167,1630168,1630169,1630170,1630171,1630172,1630173,1630174,1630175,1630177,1630178,1630180,1630181,1630182,1630183,1630184,1630188,1630190,1630191,1630192,1630193,1630194,1630195,1630197,1630198,1630200,1630201,1630202,1630205,1630208,1630209,1630214,1630215,1630217,1630219,1630224,1630227,1630228,1630230,1630231,1630233,1630235,1630240,1630245,1630249,1630256,1630264,1630267,1630314,1630322,1630526,1630527,1630528,1630529,1630530,1630532,1630533,1630535,1630537,1630538,1630540,1630541,1630543,1630544,1630547,1630549,1630550,1630551,1630553,1630556,1630557,1630558,1630559,1630560,1630561,1630563,1630567,1630568,1630570,1630572,1630578,1630579,1630580,1630581,1630583,1630586,1630587,1630591,1630595,1630596,1630598,1630613,1630625,1630631,1630678,1630686,1630688,1630792,1630846,200746,200752,200765,200768,200782,200794,201142,201143,201144,201145,201152,201565,201566,201567,201568,201569,201571,201572,201577,201580,201586,201587,201588,201599,201609,201933,201935,201937,201939,201942,201949,201950,201959,201961,201976,201980,201988,202066,202083,202324,202326,202330,202331,202334,202339,202340,202355,202357,202362,202397,202681,202683,202684,202685,202687,202689,202691,202692,202693,202694,202696,202699,202704,202709,202710,202711,202722,202738,202954,203076,203078,203081,203082,203083,203084,203085,203087,203090,203095,203107,203109,203110,203114,203115,203145,203200,203210,203457,203458,203463,203468,203469,203471,203473,203476,203482,203484,203486,203488,203490,203493,203496,203497,203500,203501,203503,203504,203507,203526,203552,203585,203648,203897,203901,203903,203914,203915,203918,203920,203924,203925,203926,203932,203935,203937,203939,203944,203952,203954,203991,203992,203994,203999,204001,204025,204060,204456,2544,2546,2730,2738,2772
0,SECONDS_PASSED,1.000000,0.966876,0.967223,0.870758,0.869803,0.010010,-0.002143,0.001460,-0.007796,-0.002475,0.010421,0.004116,-0.009340,-0.006002,-0.000136,-0.003802,0.002425,0.007172,0.002932,0.004383,0.003392,-0.001840,-0.002441,0.000836,0.000875,0.001902,-0.000769,-0.000545,-0.002115,-0.002447,0.000698,0.003227,0.004742,-0.002217,-0.003258,0.005256,0.003267,0.001260,0.009174,0.002826,-0.006019,-0.004935,0.004566,0.001266,-1.8704

In [80]:
more_results = results[~results['index'].isin(cols)][['index','HOME_VS_AWAY_PER_POSSESSION']]
more_results['index']=more_results['index'].astype(int)
player_results = more_results.merge(players,how='inner',left_on='index',right_on='PLAYER_ID')
player_results.sort_values(by='HOME_VS_AWAY_PER_POSSESSION',ascending=False,inplace=True)
player_results.head(50)

,index,HOME_VS_AWAY_PER_POSSESSION,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,TEAM_ABBREVIATION,PLAYER_ID,POSITION,HEIGHT,WEIGHT,DRAFT_YEAR,FROM_YEAR,TO_YEAR
98,1628969,0.027181,Mikal,Bridges,PHX,1628969,F,6-6,209,2018.0,2018,2021
384,203109,0.023200,Jae,Crowder,PHX,203109,F,6-6,235,2012.0,2012,2021
0,101108,0.022544,Chris,Paul,PHX,101108,G,6-0,175,2005.0,2005,2021
64,1628369,0.020960,Jayson,Tatum,BOS,1628369,F-G,6-8,210,2017.0,2017,2021
427,203935,0.019794,Marcus,Smart,BOS,203935,G,6-4,220,2014.0,2014,2021
11,1626164,0.019761,Devin,Booker,PHX,1626164,G,6-5,206,2015.0,2015,2021
181,1629661,0.019548,Cameron,Johnson,PHX,1629661,F,6-8,210,2019.0,2019,2021
411,203507,0.018782,Giannis,Antetokounmpo,MIL,203507,F,6-11,242,2013.0,2013,2021
53,1627832,0.017268,Fred,VanVleet,TOR,1627832,G,6-1,197,NaN,2016,2021
326,201588,0.017089,George,Hill,MIL,201588,G,6-4,188,2008.0,2008,2021


In [81]:
player_results.tail(50)

,index,HOME_VS_AWAY_PER_POSSESSION,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,TEAM_ABBREVIATION,PLAYER_ID,POSITION,HEIGHT,WEIGHT,DRAFT_YEAR,FROM_YEAR,TO_YEAR
170,1629644,-0.010466,KZ,Okpala,MIA,1629644,F-G,6-8,215,2019.0,2019,2021
278,1630560,-0.010468,Cam,Thomas,BKN,1630560,G,6-3,210,2021.0,2021,2021
18,1626174,-0.010522,Christian,Wood,HOU,1626174,F,6-9,214,NaN,2015,2021
389,203200,-0.010787,Justin,Holiday,SAC,203200,F-G,6-6,180,NaN,2012,2021
440,204456,-0.010813,T.J.,McConnell,IND,204456,G,6-1,190,2015.0,2015,2021
431,203952,-0.010879,Andrew,Wiggins,GSW,203952,F,6-7,197,2014.0,2014,2021
255,1630526,-0.010911,Jeremiah,Robinson-Earl,OKC,1630526,F,6-8,242,2021.0,2021,2021
299,1630678,-0.010937,Terry,Taylor,IND,1630678,F,6-5,230,NaN,2021,2021
79,1628400,-0.010973,Semi,Ojeleye,LAC,1628400,F,6-6,240,2017.0,2017,2021
39,1627756,-0.011026,Denzel,Valentine,UTA,1627756,G,6-4,220,2016.0,2016,2021


In [82]:
player_results[player_results['TEAM_ABBREVIATION']=='MEM']

,index,HOME_VS_AWAY_PER_POSSESSION,PLAYER_FIRST_NAME,PLAYER_LAST_NAME,TEAM_ABBREVIATION,PLAYER_ID,POSITION,HEIGHT,WEIGHT,DRAFT_YEAR,FROM_YEAR,TO_YEAR
111,1628991,0.013067,Jaren,Jackson Jr.,MEM,1628991,F-C,6-11,242,2018.0,2018,2021
238,1630217,0.009644,Desmond,Bane,MEM,1630217,G,6-5,215,2020.0,2020,2021
115,1629001,0.008690,De'Anthony,Melton,MEM,1629001,G,6-2,200,2018.0,2018,2021
84,1628415,0.008486,Dillon,Brooks,MEM,1628415,G-F,6-7,225,2017.0,2017,2021
2,1626145,0.006599,Tyus,Jones,MEM,1626145,G,6-0,196,2015.0,2015,2021
236,1630214,0.006425,Xavier,Tillman,MEM,1630214,F,6-8,245,2020.0,2020,2021
161,1629634,0.005162,Brandon,Clarke,MEM,1629634,F,6-8,215,2019.0,2019,2021
157,1629630,0.004757,Ja,Morant,MEM,1629630,G,6-3,174,2019.0,2019,2021
407,203500,0.004690,Steven,Adams,MEM,203500,C,6-11,265,2013.0,2013,2021
195,1629723,0.004664,John,Konchar,MEM,1629723,G,6-5,210,NaN,2019,2021
